In [ ]:
# Le dataset MNIST
from tensorflow.keras.datasets import mnist
import tensorflow as tf
from tensorflow.keras.layers import Input,Dense, Conv2D, Conv2DTranspose, MaxPooling2D, Flatten, UpSampling2D, Reshape
from tensorflow.keras.models import Model,Sequential
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import numpy as np

def MNIST_AE_disp(img_in, img_out, img_idx):

    num_img = len(img_idx)
    plt.figure(figsize=(18, 4))

    for i, image_idx in enumerate(img_idx):
        # 显示输入图像
        ax = plt.subplot(2, num_img, i + 1)
        plt.imshow(img_in[image_idx].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # 显示输出图像
        ax = plt.subplot(2, num_img, num_img + i + 1)
        plt.imshow(img_out[image_idx].reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()
    x_test=img_in[image_idx].reshape(28, 28)
    decoded_img=img_out[image_idx].reshape(28, 28)
    print(compare_psnr(x_test, decoded_img, data_range=x_test.max()-x_test.min()), ssim(x_test, decoded_img, data_range=x_test.max()-x_test.min(), multichannel=False))

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data("C:\\Users\\FRH\\Desktop\\Code\\mnist.npz")
# 正则化 [0, 255] à [0, 1]
x_train=x_train.astype('float32')/float(x_train.max())
x_test=x_test.astype('float32')/float(x_test.max())

x_train=x_train.reshape(len(x_train),x_train.shape[1], x_train.shape[2], 1)
x_test=x_test.reshape(len(x_test),x_test.shape[1], x_test.shape[2], 1)

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model

# 编码
input_img = Input(shape=(28,28,1))
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(input_img)
x = Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same')(x)
x = Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same')(x)
# x = MaxPooling2D(pool_size=(2,2))(x)
encoded = Conv2D(filters=1, kernel_size=(3,3), activation='relu', padding='same')(x)


# 解码
x = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(encoded)
# x = UpSampling2D(size=(2,2))(x)
x = Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same')(x)
x = Conv2D(filters=16, kernel_size=(3,3), activation='relu', padding='same')(x)
decoded = Conv2D(filters=1,kernel_size=(3,3), activation='sigmoid', padding='same')(x)

autoencodeur = Model(input_img, decoded)
autoencodeur.summary()

In [ ]:
autoencodeur.compile(optimizer='Adam',loss='mse')
autoencodeur.fit(x_train, x_train, batch_size=256, epochs=100)
autoencodeur.save('my_autoencoder.h5') 

In [ ]:
from tensorflow.keras.models import load_model

# 加载模型
# autoencodeur = load_model('my_autoencoder.h5')

# 挑选图片
test_images=np.array([8])
# # 预测输出图片
# decoded_img=autoencodeur.predict(x_test)
# # 显示并对比输入与输出图片
# MNIST_AE_disp(x_test, decoded_img, test_images)

model_1=Model(input_img,encoded)
test_1=model_1.predict(x_test)
# print(test_1[8].shape,test_1[8].reshape(784))

data_test=test_1[8].reshape(784)

# 归一化数组
normalized_array = (data_test - data_test.min()) / (data_test.max() - data_test.min())

# 将每个数组元素映射到0到15之间的整数
quantized_values = np.round(normalized_array * 15).astype(int)

# 将这些整数转换为4位的二进制表示
binary_representation = np.array(
    [np.binary_repr(v, width=4) for v in quantized_values])

# 将二进制表示转换为NumPy数组中的每个位作为一个数字
binary_array = np.array([list(map(int, list(b)))
                        for b in binary_representation]).reshape(-1, 4)

# 将二进制数组转换为字符串，每个数字之间用空格分隔，每行对应一个原始的4位二进制数
binary_array_str = '\n'.join([' '.join(map(str, row)) for row in binary_array])

# 指定要保存的文件路径
file_path = 'C:/Users/FRH/Desktop/Code/data_1.txt'

# 将字符串写入文本文件
with open(file_path, 'w') as file:
    file.write(binary_array_str)

print(f"二进制数组已保存到文本文件：{file_path}")

# 定义还原函数
def restore_original_values(binary_array, min_value, max_value):
    # 将二进制数组转换回十进制整数
    decimal_values = np.array([int("".join(map(str, bits)), 2)
                              for bits in binary_array])

    # 还原归一化的值
    normalized_values = decimal_values / 15

    # 还原到原始范围
    original_values = normalized_values * (max_value - min_value) + min_value
    return original_values

import numpy as np
with open('C:/Users/FRH/Desktop/Code/data_2.txt') as f:
    line=f.readline()
    data_array=[]
    while line:
        num=list(map(int,line.split(' ')))
        data_array.append(num)
        line=f.readline()
    data_array=np.array(data_array)

# 使用还原函数还原原始数组
restored_values1 = restore_original_values(
    binary_array, data_test.min(), data_test.max())
restored_values2 = restore_original_values(
    data_array, data_test.min(), data_test.max())

# import scipy.signal as signal
# restored_values_d=restored_values1-restored_values2
# # print(restored_values_d.shape,restored_values_d)
# restored_values=signal.medfilt(restored_values_d,3)+restored_values2 #一维中值滤波

# print("原始数组:", data_test)
# print("归一化数组:", normalized_array)
# print("量化值:", quantized_values)
# print("二进制表示:", binary_representation)
# print("二进制数组:", binary_array)
# print("还原的原始值:", restored_values)

model_2=Model(encoded,decoded)
temp_1=restored_values1.reshape(28,28,1)
repeated_array_1 = np.repeat(temp_1[np.newaxis, :, :, :], 10000, axis=0)
test_2=model_2.predict(repeated_array_1)
MNIST_AE_disp(x_test, test_2, test_images)

model_3=Model(encoded,decoded)
temp_2=restored_values2.reshape(28,28,1)
repeated_array_2 = np.repeat(temp_2[np.newaxis, :, :, :], 10000, axis=0)
test_3=model_3.predict(repeated_array_2)
MNIST_AE_disp(x_test, test_3, test_images)

import imageio
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import numpy as np

image_true = imageio.imread(
    'C:\\Users\\FRH\\Desktop\\Code\\2023凌特杯\\Project\\SIMULINK\\com_tx.jpg')
image_test = imageio.imread(
    'C:\\Users\\FRH\\Desktop\\Code\\2023凌特杯\\Project\\SIMULINK\\com_rx.jpg')

PSNR_value = psnr(image_true, image_test, data_range=image_true.max()-image_test.min())
ssim_values = []
for i in range(image_true.shape[2]):
    ssim_value = ssim(image_true[:,:,i], image_test[:,:,i], data_range=image_true.max()-image_test.min(), multichannel=False)
    ssim_values.append(ssim_value)
SSIM_value = np.mean(ssim_values)

print(f"PSNR value: {PSNR_value} dB")
print(f"SSIM value: {SSIM_value} ")
